In [1]:
from pennylane import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
import torch

from feature_map import *
from ansatz import *
from cost_function import *
from loss_function import *

In [2]:
def generate_chebyshev_grid(n, a=0, b=0.9, requires_grad=True):
    k = torch.arange(n)
    chebyshev_nodes = torch.cos(torch.pi * (2*k + 1) / (2*n))  #standard Chebyshev nodes in [-1,1]
    scaled_nodes = ((chebyshev_nodes + 1) / 2) * (b - a) + a  #rescale to [a, b]
    
    return scaled_nodes.requires_grad_(requires_grad=requires_grad)

N_POINTS = 10
x = generate_chebyshev_grid(N_POINTS, requires_grad=False).detach().numpy()
print(x)

[0.8944597  0.85095286 0.7681981  0.6542957  0.5203955  0.37960446
 0.24570423 0.13180195 0.04904703 0.00554024]


In [3]:
#x = np.linspace(0, 0.9, 10)
#print(x)
#np.random.seed(42)
#theta_init = np.array(2*np.pi*np.random.random(num_qubits*d*3), requires_grad=True)
qadence_init = [0.05815449, 0.9552357, 0.84251973, 0.62340308, 0.03134017, 0.25185522, 0.06291017, 0.92875258, 0.87050104, 0.437279, 0.04036647, 0.23340819, 0.12358607, 0.08354337, 0.37753296, 0.07473488, 0.93190879, 0.2256651, 0.05258016, 0.13264067, 0.61243659, 0.68343022, 0.15212583, 0.3175678, 0.5261719, 0.1570535, 0.08810978, 0.31223868, 0.26504491, 0.61440828, 0.47678475, 0.37536703, 0.70118189, 0.30905878, 0.13043349, 0.59777135]

theta_init = np.array(qadence_init, requires_grad=True)
print(theta_init)

[0.05815449 0.9552357  0.84251973 0.62340308 0.03134017 0.25185522
 0.06291017 0.92875258 0.87050104 0.437279   0.04036647 0.23340819
 0.12358607 0.08354337 0.37753296 0.07473488 0.93190879 0.2256651
 0.05258016 0.13264067 0.61243659 0.68343022 0.15212583 0.3175678
 0.5261719  0.1570535  0.08810978 0.31223868 0.26504491 0.61440828
 0.47678475 0.37536703 0.70118189 0.30905878 0.13043349 0.59777135]


### f(x)

In [4]:
num_qubits = 6
d = 2

def total_magnetization(num_qubits):
    return sum(qml.PauliZ(i) for i in range(num_qubits))
def create_circuit(x, num_qubits, d, theta, label):

    U(x, num_qubits, label)
    qml.Barrier()
    HEA(theta, num_qubits, d)
    cost_op = total_magnetization(num_qubits) 
   
    return qml.expval(cost_op)

In [5]:
functions = []
derivatives = []
print(x)
for i in range(len(x)):

    func = f(x[i], num_qubits, d, theta_init)
    print("F(x): ",func)
    f0 = f(0, num_qubits, d, theta_init)
    print("F(0): ",f0)
    functions.append(func + (1 - f0)) #FLOATING BOUNDARY HANDLING!!! 

    deriv = df(x[i], num_qubits, d, theta_init)
    derivatives.append(deriv)

print(np.array(functions))
print(np.array(derivatives))

[0.8944597  0.85095286 0.7681981  0.6542957  0.5203955  0.37960446
 0.24570423 0.13180195 0.04904703 0.00554024]
F(x):  1.1673947095645127
F(0):  -0.5952419788090493
F(x):  0.8658625023728586
F(0):  -0.5952419788090493
F(x):  0.26096701252118704
F(0):  -0.5952419788090493
F(x):  0.362124873985089
F(0):  -0.5952419788090493
F(x):  0.4366071591504959
F(0):  -0.5952419788090493
F(x):  -0.20863408369786623
F(0):  -0.5952419788090493
F(x):  -0.49510644676582294
F(0):  -0.5952419788090493
F(x):  -0.4556332979243217
F(0):  -0.5952419788090493
F(x):  -0.5015333855700024
F(0):  -0.5952419788090493
F(x):  -0.5827048206133318
F(0):  -0.5952419788090493
[2.76263669 2.46110448 1.85620899 1.95736685 2.03184914 1.3866079
 1.10013553 1.13960868 1.09370859 1.01253716]
[ 5.6633779   8.17349226  4.22752813 -3.49751132  3.19478177  4.17595462
  0.21604163 -0.22001979  1.44179529  2.22805335]


In [6]:
loss = loss_function(x, num_qubits, d, theta_init, 0, 0, 0)
print(loss)

76.64045950401587
